In [1]:
from abc import ABC, abstractmethod

# Singleton Pattern: Ensures only one instance of the class exists.
class BankControlSystem:
    _instance = None

    @staticmethod
    def get_instance():
        # Ensures only one instance of the class exists.
        if BankControlSystem._instance is None:
            BankControlSystem()
        return BankControlSystem._instance

    def __init__(self):
        if BankControlSystem._instance is not None:
            raise Exception("This class is a singleton!")
        else:
            BankControlSystem._instance = self
            self.accounts = {}

    def add_account(self, account):
        # Add a bank account to the system.
        self.accounts[account.account_number] = account

    def get_account(self, account_number):
        # Retrieve a bank account by account number.
        return self.accounts.get(account_number)

# Abstract base class for bank accounts.
class BankAccount(ABC):
    def __init__(self, account_number, balance=0):
        self.account_number = account_number
        self.balance = balance

    @abstractmethod
    def deposit(self, amount):
        # Abstract method for deposit.
        pass

    @abstractmethod
    def withdraw(self, amount):
        # Abstract method for withdrawal.
        pass

    @abstractmethod
    def get_balance(self):
        # Abstract method for getting the balance.
        pass

# Concrete class for savings accounts.
class SavingsAccount(BankAccount):
    def deposit(self, amount):
        self.balance += amount
        print(f"Deposited ${amount} to Savings Account {self.account_number}. New balance: ${self.balance}")

    def withdraw(self, amount):
        if amount <= self.balance:
            self.balance -= amount
            print(f"Withdrew ${amount} from Savings Account {self.account_number}. New balance: ${self.balance}")
        else:
            print("Insufficient funds")

    def get_balance(self):
        return self.balance

# Concrete class for checking accounts.
class CheckingAccount(BankAccount):
    def deposit(self, amount):
        self.balance += amount
        print(f"Deposited ${amount} to Checking Account {self.account_number}. New balance: ${self.balance}")

    def withdraw(self, amount):
        if amount <= self.balance:
            self.balance -= amount
            print(f"Withdrew ${amount} from Checking Account {self.account_number}. New balance: ${self.balance}")
        else:
            print("Insufficient funds")

    def get_balance(self):
        return self.balance

In [2]:
# Factory Pattern: Provides a method to create instances of different types of bank accounts.
class BankAccountFactory:
    @staticmethod
    def create_account(account_type, account_number):
        # Create and return a specific type of bank account.
        if account_type == "Savings":
            return SavingsAccount(account_number)
        elif account_type == "Checking":
            return CheckingAccount(account_number)
        else:
            raise ValueError("Invalid account type")

In [3]:
# Observer Pattern: Allows the transaction system to notify its observers about transactions.
class Subject:
    def __init__(self):
        self._observers = []

    def attach(self, observer):
        # Attach an observer.
        self._observers.append(observer)

    def detach(self, observer):
        # Detach an observer.
        self._observers.remove(observer)

    def notify(self, message):
        # Notify all observers.
        for observer in self._observers:
            observer.update(message)

# Abstract base class for observers.
class Observer(ABC):
    @abstractmethod
    def update(self, message):
        # Abstract method for update.
        pass

# Concrete observer class for users.
class UserObserver(Observer):
    def update(self, message):
        print(f"Notification: {message}")

# Transaction system class implementing the observer pattern.
class TransactionSystem(Subject):
    def record_transaction(self, message):
        # Record a transaction and notify observers.
        self.notify(message)

In [4]:
# Decorator Pattern: Adds additional functionality (e.g., Overdraft Protection) to bank accounts dynamically.
class AccountDecorator(BankAccount):
    def __init__(self, decorated_account):
        self.decorated_account = decorated_account

    def deposit(self, amount):
        # Delegate deposit to the decorated account.
        self.decorated_account.deposit(amount)

    def withdraw(self, amount):
        # Delegate withdrawal to the decorated account.
        self.decorated_account.withdraw(amount)

    def get_balance(self):
        # Delegate get balance to the decorated account.
        return self.decorated_account.get_balance()

# Concrete decorator class for overdraft protection.
class OverdraftProtection(AccountDecorator):
    def __init__(self, decorated_account, overdraft_limit):
        super().__init__(decorated_account)
        self.overdraft_limit = overdraft_limit

    def withdraw(self, amount):
        # Withdraw with overdraft protection.
        if amount <= self.decorated_account.get_balance() + self.overdraft_limit:
            self.decorated_account.withdraw(amount)
            print(f"Overdraft protection applied. New balance: ${self.decorated_account.get_balance()}")
        else:
            print("Overdraft limit exceeded")

In [5]:
# Strategy Pattern: Allows dynamic selection of different interest calculation strategies.
class InterestStrategy(ABC):
    @abstractmethod
    def calculate_interest(self, balance):
        # Abstract method to calculate interest.
        pass

# Concrete class for simple interest strategy.
class SimpleInterestStrategy(InterestStrategy):
    def calculate_interest(self, balance):
        return balance * 0.01

# Concrete class for compound interest strategy.
class CompoundInterestStrategy(InterestStrategy):
    def calculate_interest(self, balance):
        return balance * ((1 + 0.01)**12 - 1)

# Concrete class for bank accounts with interest.
class BankAccountWithInterest(BankAccount):
    def __init__(self, account_number, balance=0, interest_strategy=None):
        super().__init__(account_number, balance)
        self.interest_strategy = interest_strategy

    def deposit(self, amount):
        self.balance += amount
        print(f"Deposited ${amount} to Account {self.account_number}. New balance: ${self.balance}")

    def withdraw(self, amount):
        if amount <= self.balance:
            self.balance -= amount
            print(f"Withdrew ${amount} from Account {self.account_number}. New balance: ${self.balance}")
        else:
            print("Insufficient funds")

    def get_balance(self):
        return self.balance

    def set_interest_strategy(self, strategy):
        # Set the interest strategy.
        self.interest_strategy = strategy

    def apply_interest(self):
        # Apply the interest strategy.
        if self.interest_strategy:
            interest = self.interest_strategy.calculate_interest(self.balance)
            self.balance += interest
            print(f"Applied interest: ${interest}. New balance: ${self.balance}")
        else:
            print("No interest strategy set")

In [6]:
# Command Pattern: Encapsulates requests as objects, allowing for parameterization of clients with queues, requests, and operations.
class Command(ABC):
    @abstractmethod
    def execute(self):
        # Abstract method to execute the command.
        pass

# Concrete command class for deposit.
class DepositCommand(Command):
    def __init__(self, account, amount):
        self.account = account
        self.amount = amount

    def execute(self):
        # Execute deposit.
        self.account.deposit(self.amount)

# Concrete command class for withdrawal.
class WithdrawCommand(Command):
    def __init__(self, account, amount):
        self.account = account
        self.amount = amount

    def execute(self):
        # Execute withdrawal.
        self.account.withdraw(self.amount)

# Invoker class to execute commands.
class Invoker:
    def __init__(self):
        self.commands = []

    def add_command(self, command):
        # Add a command to the list.
        self.commands.append(command)

    def execute_commands(self):
        # Execute all commands in the list.
        for command in self.commands:
            command.execute()
        self.commands = []

In [7]:
# Adapter Pattern: Allows incompatible interfaces (ThirdPartyPaymentSystem) to work together.
class ThirdPartyPaymentSystem:
    def make_payment(self, amount):
        # Make a payment through third-party system.
        print(f"Payment of ${amount} made through third-party system")

# Adapter class to integrate third-party payment system.
class PaymentAdapter:
    def __init__(self, third_party_system):
        self.third_party_system = third_party_system

    def pay(self, amount):
        # Adapt the third-party payment system to the expected interface.
        self.third_party_system.make_payment(amount)

# User class with roles.
class User:
    def __init__(self, name, role):
        self.name = name
        self.role = role

    def has_permission(self, account):
        # Simple permission logic based on role.
        if self.role == "Admin":
            return True
        elif self.role == "User" and isinstance(account, CheckingAccount):
            return True
        return False

In [8]:
# Proxy Pattern: Controls access to a bank account based on the user's permissions.
class AccountProxy:
    def __init__(self, real_account, user):
        self.real_account = real_account
        self.user = user

    def deposit(self, amount):
        # Proxy method for deposit with permission check.
        if self.user.has_permission(self.real_account):
            self.real_account.deposit(amount)
        else:
            print("Access Denied: Insufficient permissions")

    def withdraw(self, amount):
        # Proxy method for withdrawal with permission check.
        if self.user.has_permission(self.real_account):
            self.real_account.withdraw(amount)
        else:
            print("Access Denied: Insufficient permissions")

    def get_balance(self):
        # Proxy method for getting balance with permission check.
        if self.user.has_permission(self.real_account):
            return self.real_account.get_balance()
        else:
            print("Access Denied: Insufficient permissions")
            return None

In [9]:
# Main script
if __name__ == "__main__":
    # Initialize bank system and transaction system.
    bank_system = BankControlSystem.get_instance()
    transaction_system = TransactionSystem()
    observer = UserObserver()
    transaction_system.attach(observer)
    invoker = Invoker()

    while True:
        # Display menu.
        print("\nMenu:")
        print("1. Create Account")
        print("2. Add Overdraft Protection")
        print("3. Apply Interest to Savings Account")
        print("4. Perform Transactions")
        print("5. Use Third-Party Payment System")
        print("6. Set Up Proxy Users")
        print("7. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            # Create a new account.
            print("Creating accounts...")
            account_type = input("Enter account type (Savings/Checking): ")
            account_number = input("Enter account number: ")
            account = BankAccountFactory.create_account(account_type, account_number)
            bank_system.add_account(account)
            print(f"{account_type} account with number {account_number} created.")
        
        elif choice == "2":
            # Add overdraft protection to a checking account.
            print("Adding overdraft protection to the checking account...")
            account_number = input("Enter checking account number: ")
            overdraft_limit = float(input("Enter overdraft limit: "))
            account = bank_system.get_account(account_number)
            if account:
                overdraft_checking = OverdraftProtection(account, overdraft_limit)
                print(f"Overdraft protection of {overdraft_limit} added to account {account_number}.")
            else:
                print("Account not found.")
        
        elif choice == "3":
            # Apply interest to a savings account.
            print("Applying interest to the savings account...")
            account_number = input("Enter savings account number: ")
            account = bank_system.get_account(account_number)
            if account:
                savings_with_interest = BankAccountWithInterest(account_number, account.get_balance())
                interest_type = input("Enter interest type (simple/compound): ")
                if interest_type == "simple":
                    strategy = SimpleInterestStrategy()
                elif interest_type == "compound":
                    strategy = CompoundInterestStrategy()
                else:
                    print("Invalid interest type.")
                    continue
                savings_with_interest.set_interest_strategy(strategy)
                savings_with_interest.apply_interest()
                print(f"Interest applied to account {account_number}. New balance: {savings_with_interest.get_balance()}")
            else:
                print("Account not found.")
        
        elif choice == "4":
            # Perform deposit and withdrawal transactions.
            print("Performing transactions...")
            account_number = input("Enter account number: ")
            account = bank_system.get_account(account_number)
            if account:
                deposit_amount = float(input("Enter deposit amount: "))
                withdraw_amount = float(input("Enter withdrawal amount: "))
                deposit_command = DepositCommand(account, deposit_amount)
                withdraw_command = WithdrawCommand(account, withdraw_amount)
                invoker.add_command(deposit_command)
                invoker.add_command(withdraw_command)
                invoker.execute_commands()
                print(f"Transactions completed for account {account_number}. New balance: {account.get_balance()}")
            else:
                print("Account not found.")
        
        elif choice == "5":
            # Use third-party payment system.
            print("Using third-party payment system...")
            third_party_system = ThirdPartyPaymentSystem()
            payment_adapter = PaymentAdapter(third_party_system)
            amount = float(input("Enter amount to pay using third-party system: "))
            payment_adapter.pay(amount)
        
        elif choice == "6":
            # Set up proxy users.
            print("Setting up proxy users...")
            admin_name = input("Enter admin user name: ")
            user_name = input("Enter regular user name: ")
            admin_user = User(admin_name, "Admin")
            regular_user = User(user_name, "User")
            account_number_admin = input("Enter account number for admin: ")
            account_number_user = input("Enter account number for user: ")
            admin_account = bank_system.get_account(account_number_admin)
            user_account = bank_system.get_account(account_number_user)
            if admin_account and user_account:
                admin_proxy = AccountProxy(admin_account, admin_user)
                user_proxy = AccountProxy(user_account, regular_user)
                admin_deposit = float(input("Enter amount for admin to deposit: "))
                user_deposit = float(input("Enter amount for user to deposit: "))
                user_withdraw = float(input("Enter amount for user to withdraw: "))
                admin_proxy.deposit(admin_deposit)
                user_proxy.deposit(user_deposit)
                user_proxy.withdraw(user_withdraw)
                print(f"Admin balance: {admin_proxy.get_balance()}")
                print(f"User balance: {user_proxy.get_balance()}")
            else:
                print("Account not found.")
        
        elif choice == "7":
            # Exit the program.
            print("Exiting...")
            break
        
        else:
            print("Invalid choice. Please try again.")


Menu:
1. Create Account
2. Add Overdraft Protection
3. Apply Interest to Savings Account
4. Perform Transactions
5. Use Third-Party Payment System
6. Set Up Proxy Users
7. Exit
Enter your choice: 1
Creating accounts...
Enter account type (Savings/Checking): Savings
Enter account number: 123456
Savings account with number 123456 created.

Menu:
1. Create Account
2. Add Overdraft Protection
3. Apply Interest to Savings Account
4. Perform Transactions
5. Use Third-Party Payment System
6. Set Up Proxy Users
7. Exit
Enter your choice: 3
Applying interest to the savings account...
Enter savings account number: 123456
Enter interest type (simple/compound): simple
Applied interest: $0.0. New balance: $0.0
Interest applied to account 123456. New balance: 0.0

Menu:
1. Create Account
2. Add Overdraft Protection
3. Apply Interest to Savings Account
4. Perform Transactions
5. Use Third-Party Payment System
6. Set Up Proxy Users
7. Exit
Enter your choice: 4
Performing transactions...
Enter account